In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import time
import dask.dataframe as dd
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive

from sklearn.preprocessing import normalize

In [ ]:
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [ ]:
col_name = ["User_ID", "Stream_ID", "Streamer_username", "Time_start", "Time_stop"]
df = pd.read_csv('/content/mydrive/MyDrive/100k_a.csv',header = None, names = col_name)
df.head()

,User_ID,Stream_ID,Streamer_username,Time_start,Time_stop
0,1,33842865744,mithrain,154,156
1,1,33846768288,alptv,166,169
2,1,33886469056,mithrain,587,588
3,1,33887624992,wtcn,589,591
4,1,33890145056,jrokezftw,591,594


In [ ]:
row_count = len(df)
print("index:", row_count)

index: 3051733


In [ ]:
df["rating"] = df["Time_stop"] - df["Time_start"]
df.head( 3051735)

,User_ID,Stream_ID,Streamer_username,Time_start,Time_stop,rating
0,1,33842865744,mithrain,154,156,2
1,1,33846768288,alptv,166,169,3
2,1,33886469056,mithrain,587,588,1
3,1,33887624992,wtcn,589,591,2
4,1,33890145056,jrokezftw,591,594,3
...,...,...,...,...,...,...
3051728,100000,34167930576,mckytv,3582,3583,1
3051729,100000,34168288656,natehill,3582,3583,1
3051730,100000,34169594512,fortnite,3582,3584,2
3051731,100000,34180223072,highdistortion,3709,3711,2


In [ ]:
sampled_df = df.iloc[:1000]

In [ ]:
sampled_df.head(1000)

,User_ID,Stream_ID,Streamer_username,Time_start,Time_stop,rating
0,1,33842865744,mithrain,154,156,2
1,1,33846768288,alptv,166,169,3
2,1,33886469056,mithrain,587,588,1
3,1,33887624992,wtcn,589,591,2
4,1,33890145056,jrokezftw,591,594,3
...,...,...,...,...,...,...
995,40,34205829536,tamazarasi,3967,3969,2
996,40,34206069344,xbt306x,3966,3967,1
997,40,34206313296,teenage_19,3966,3967,1
998,40,34215890048,kalrto,4069,4070,1


유저와 유저사이 추천

In [ ]:
# Assuming sampled_df is your DataFrame containing User_ID, Streamer_username, and rating columns

# Assign numeric indexes to User ID and Stream ID
user_ids, user_names = pd.factorize(sampled_df['User_ID'])
stream_ids, stream_names = pd.factorize(sampled_df['Streamer_username'])

ratings_data = sampled_df['rating'].values.astype(np.float128)

# Create csr_matrix
user_item_matrix = csr_matrix((sampled_df['rating'].values,
                               (user_ids, stream_ids)),
                               shape=(len(np.unique(user_ids)), len(np.unique(stream_ids)))).toarray()

user_item_matrix = pd.DataFrame(user_item_matrix, index=user_names, columns=stream_names)









non_zero_count = (user_item_matrix.to_numpy() != 0).sum()

# Calculate the total number of elements
total_elements = user_item_matrix.size

# Calculation of scarcity ratio
sparsity_ratio = non_zero_count / total_elements
print(f"희소 비율: {sparsity_ratio}")


# Normalize user-item matrix
normalized_user_item_matrix = normalize(user_item_matrix, axis=1, norm='l2')

print(normalized_user_item_matrix)

total_sum = np.sum(normalized_user_item_matrix)
print("행렬의 총합:", total_sum)
#cosine similarity


from sklearn.metrics.pairwise import cosine_similarity


user_similarity_matrix = cosine_similarity(normalized_user_item_matrix)


# similarity
similarity = pd.DataFrame(user_similarity_matrix, index=user_names, columns=user_names)
similarity




희소 비율: 0.028216704288939052
[[0.25197632 0.07559289 0.27717395 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.03817573 0.04581087 0.01527029]]
행렬의 총합: 92.3563786050821


,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
1,1.000000,0.000000,0.0,0.459534,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
4,0.459534,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.0,0.007876,0.0,...,0.074804,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
5,0.000000,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
6,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
7,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000450,0.0,0.000000,0.000000,0.0
8,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
9,0.000000,0.000000,0.0,0.007876,0.0,0.0,0.000000,0.0,1.000000,0.0,...,0.091550,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
10,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,1.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0


In [ ]:

similarities = cosine_similarity(normalized_user_item_matrix.T)  # 열(특성)을 기준으로 유사도 계산

# Determine the index of where you want to fill the missing values
missing_indices = np.argwhere(np.isnan(normalized_user_item_matrix))

# Run an iteration to populate missing values
for index in missing_indices:
    row_idx, col_idx = index
    # Run iteration to fill in missing values# To predict missing values in the current location, predict using values from characteristics (column) similar to that location
    prediction = np.dot(similarities[col_idx], normalized_user_item_matrix[:, row_idx]) / np.sum(np.abs(similarities[col_idx]))
    normalized_user_item_matrix[row_idx, col_idx] = prediction

print("결과:")
print(normalized_user_item_matrix)
total_sum = np.sum(normalized_user_item_matrix)
print("행렬의 총합:", total_sum)
#cosine similarity


결과:
[[0.25197632 0.07559289 0.27717395 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.03817573 0.04581087 0.01527029]]
행렬의 총합: 92.3563786050821


비슷한 유형의 사람 찾기

In [ ]:
def get_streamer_user_similarity_all_users():
    recommended_streamers = {}
    for user_name in user_names:
        user_id = user_names.get_loc(user_name)
        # Select the row corresponding to user_id in the similarity matrix, and select the 5 most similar users except yourself
        similar_users = similarity.iloc[user_id].drop(user_name, errors='ignore').sort_values(ascending=False)[:5]

        # Choose the most watched streamer among streamers watched by similar users
        similar_users_streamers = user_item_matrix.loc[similar_users.index]
        top_streamer = similar_users_streamers.sum().idxmax()
        recommended_streamers[user_name] = top_streamer

    return recommended_streamers

# Find recommended streamers for all users
all_recommended_streamers = get_streamer_user_similarity_all_users()
print("사용자와 유형의 비슷한 시청자 추천 스트리머:")

import pprint


pp = pprint.PrettyPrinter(indent=4)
pp.pprint(all_recommended_streamers)

사용자와 유형의 비슷한 시청자 추천 스트리머:
{   1: 'kendinemuzisyen',
    2: 'asiagodtonegg3be0',
    3: 'kendinemuzisyen',
    4: 'kendinemuzisyen',
    5: 'kendinemuzisyen',
    6: 'kendinemuzisyen',
    7: 'looksam',
    8: 'kendinemuzisyen',
    9: 'tfue',
    10: 'kendinemuzisyen',
    11: 'tfue',
    12: 'thedarkness',
    13: 'kendinemuzisyen',
    14: 'kendinemuzisyen',
    15: 'h2p_gucio',
    16: 'kendinemuzisyen',
    17: 'tfblade',
    18: 'kendinemuzisyen',
    19: 'kendinemuzisyen',
    20: 'kendinemuzisyen',
    21: 'kendinemuzisyen',
    22: 'tfblade',
    23: 'kimsoyea',
    24: 'kendinemuzisyen',
    25: 'kimsoyea',
    26: 'thedarkness',
    27: 'tfblade',
    28: 'kendinemuzisyen',
    29: 'kendinemuzisyen',
    30: 'kendinemuzisyen',
    31: 'tfue',
    32: 'tfblade',
    33: 'kendinemuzisyen',
    34: 'riotgames',
    35: 'kendinemuzisyen',
    36: 'kimsoyea',
    37: 'kendinemuzisyen',
    38: 'tfblade',
    39: 'tfblade',
    40: 'kendinemuzisyen'}
